In [61]:
import importlib
import Alpha
importlib.reload(Alpha)
from Alpha import merge_df, completed

In [62]:
train = pd.read_csv("C:\\Users\\solba\\dacon-project\\data\\raw\\train.csv")
test = pd.read_csv("C:\\Users\\solba\\dacon-project\\data\\raw\\test.csv")

In [ ]:
train = merge_df(train)

In [64]:
data

,ID,school1,job,nationality,High Tech,Data Friendly,Others,hope_for_group,desired_career_path,incumbents_level,incumbents_lecture,incumbents_company_level,incumbents_lecture_type,incumbents_lecture_scale,count,time_input,want_count
0,TRAIN_000,22,0,1,0,1,1,1,0,0,0,16,1,2,3,2,8
1,TRAIN_001,1,0,1,0,0,1,0,3,1,7,6,0,5,1,3,10
2,TRAIN_002,27,0,1,0,0,1,0,0,1,5,4,0,5,2,10,12
3,TRAIN_003,1,0,1,0,0,1,1,3,1,7,6,1,5,3,2,7
4,TRAIN_004,16,0,1,1,0,1,0,3,1,5,4,1,5,2,2,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
743,TRAIN_743,13,0,1,1,1,0,0,3,1,5,4,1,5,3,3,6
744,TRAIN_744,43,0,1,1,0,0,1,3,1,5,4,1,2,3,3,6
745,TRAIN_745,11,0,1,0,1,1,0,0,0,7,16,1,2,1,1,5
746,TRAIN_746,30,0,1,1,1,0,2,3,1,7,4,1,5,1,2,8


In [65]:
Y = completed(train, data)

In [66]:
Y

,ID,school1,job,nationality,High Tech,Data Friendly,Others,hope_for_group,desired_career_path,incumbents_level,incumbents_lecture,incumbents_company_level,incumbents_lecture_type,incumbents_lecture_scale,count,time_input,want_count,completed
0,TRAIN_000,22,0,1,0,1,1,1,0,0,0,16,1,2,3,2,8,0
1,TRAIN_001,1,0,1,0,0,1,0,3,1,7,6,0,5,1,3,10,0
2,TRAIN_002,27,0,1,0,0,1,0,0,1,5,4,0,5,2,10,12,0
3,TRAIN_003,1,0,1,0,0,1,1,3,1,7,6,1,5,3,2,7,1
4,TRAIN_004,16,0,1,1,0,1,0,3,1,5,4,1,5,2,2,11,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
743,TRAIN_743,13,0,1,1,1,0,0,3,1,5,4,1,5,3,3,6,0
744,TRAIN_744,43,0,1,1,0,0,1,3,1,5,4,1,2,3,3,6,1
745,TRAIN_745,11,0,1,0,1,1,0,0,0,7,16,1,2,1,1,5,1
746,TRAIN_746,30,0,1,1,1,0,2,3,1,7,4,1,5,1,2,8,1


In [5]:
from __future__ import annotations
import pandas as pd
import importlib
import Alpha
importlib.reload(Alpha)
from Alpha import merge_df, completed

import os
import numpy as np
import pandas as pd

from catboost import CatBoostClassifier, Pool
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, f1_score, accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import roc_auc_score, f1_score, accuracy_score, precision_score, recall_score

train = pd.read_csv("/Users/parksung-cheol/Desktop/dacon-project/data/raw/train.csv")
test = pd.read_csv("/Users/parksung-cheol/Desktop/dacon-project/data/raw/test.csv")

train_df = merge_df(train)
test_df = merge_df(test)

train_df = completed(train, train_df)

"""
CatBoost binary classification (completed: 0/1)
- Mixed types: categorical(str/ID-like) + numeric(int)
- Stratified K-Fold CV (AUC)
- Out-of-fold probability + threshold tuning for best F1
- Final training on full data + prediction on test

Usage:
  1) Put train.csv / test.csv in DATA_DIR
  2) Adjust FILE names if needed
  3) Run

Notes:
- CatBoost handles categorical features if you pass cat_features indices (or names via Pool).
- We will treat the following as categorical:
  school1, job, nationality, High Tech, Data Friendly, Others,
  hope_for_group, desired_career_path,
  incumbents_level, incumbents_lecture, incumbents_company_level,
  incumbents_lecture_type, incumbents_lecture_scale
- Numeric:
  count, time_input, want_count
"""



# --- after you build train_df/test_df and add completed ---
train = train_df.copy()
test = test_df.copy()

# ✅ 핵심: ID로 정렬해서 row alignment 고정
train = train.sort_values("ID").reset_index(drop=True)
test  = test.sort_values("ID").reset_index(drop=True)

train_df, val_df = train_test_split(train, test_size=0.2, random_state=42)

train_df.reset_index(drop=True, inplace=True)
val_df.reset_index(drop=True, inplace=True)

cat_cols = [
    "school1","job","nationality","High Tech","Data Friendly","Others",
    "hope_for_group","desired_career_path",
    "incumbents_level","incumbents_lecture","incumbents_company_level",
    "incumbents_lecture_type","incumbents_lecture_scale",
]
num_cols = ["count","time_input","want_count"]

encoded_categoric_cols = []
target = train_df["completed"].astype(int)

scaler = MinMaxScaler()
train_df[num_cols] = scaler.fit_transform(train_df[num_cols])

In [7]:
train_df[cat_cols]

,school1,job,nationality,High Tech,Data Friendly,Others,hope_for_group,desired_career_path,incumbents_level,incumbents_lecture,incumbents_company_level,incumbents_lecture_type,incumbents_lecture_scale
0,1,0,1,1,0,0,2,3,1,7,6,1,2
1,13,0,1,0,0,1,1,3,1,7,16,1,5
2,20,0,1,1,0,1,0,3,1,7,4,1,2
3,63,0,1,1,0,0,0,3,1,5,4,1,5
4,22,0,1,0,1,0,0,3,0,7,6,1,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
593,27,0,1,0,1,0,2,3,1,7,6,1,5
594,73,0,1,1,0,0,1,3,1,5,4,1,5
595,22,0,1,0,1,0,2,0,1,7,6,1,5
596,54,0,1,1,0,0,0,3,1,7,4,0,2


In [6]:
train_df

,ID,school1,job,nationality,High Tech,Data Friendly,Others,hope_for_group,desired_career_path,incumbents_level,incumbents_lecture,incumbents_company_level,incumbents_lecture_type,incumbents_lecture_scale,count,time_input,want_count,completed
0,TRAIN_593,1,0,1,1,0,0,2,3,1,7,6,1,2,0.000000,0.083333,0.277778,0
1,TRAIN_131,13,0,1,0,0,1,1,3,1,7,16,1,5,0.000000,0.083333,0.388889,0
2,TRAIN_044,20,0,1,1,0,1,0,3,1,7,4,1,2,0.142857,0.166667,0.388889,1
3,TRAIN_070,63,0,1,1,0,0,0,3,1,5,4,1,5,0.000000,0.083333,0.333333,1
4,TRAIN_670,22,0,1,0,1,0,0,3,0,7,6,1,5,0.000000,0.083333,0.055556,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
593,TRAIN_071,27,0,1,0,1,0,2,3,1,7,6,1,5,0.000000,0.083333,0.277778,0
594,TRAIN_106,73,0,1,1,0,0,1,3,1,5,4,1,5,0.428571,0.166667,0.166667,1
595,TRAIN_270,22,0,1,0,1,0,2,0,1,7,6,1,5,0.428571,0.083333,0.111111,1
596,TRAIN_435,54,0,1,1,0,0,0,3,1,7,4,0,2,0.000000,0.083333,0.333333,0


0      0
1      0
2      0
3      1
4      0
      ..
743    0
744    1
745    1
746    1
747    1
Name: completed, Length: 748, dtype: int64